In [28]:
import pandas as pd

In [69]:
path_to_file = '5_data.csv.zip'

In [70]:
retail = pd.read_csv(path_to_file, compression='zip', encoding='ISO-8859-1')

In [71]:
retail_columns = retail.columns

In [72]:
# whether there are duplicate observations in the data
# What is their number? Delete them
retail.duplicated().sum()

5268

In [73]:
retail = retail.drop_duplicates()

In [74]:
# If the user canceled the order, the transaction number (InvoiceNo) is preceded by C (canceled)
# How many transactions were canceled by users in total?
retail.InvoiceNo.str.startswith('C').sum()

9251

In [75]:
# filter the data and leave in retail only those orders where Quantity > 0
# indicate the number of remaining rows.
retail.loc[retail.Quantity > 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [101]:
# Count the number of orders for each user (CustomerID) from Germany
# Leave only those who made more than N transactions (InvoiceNo), where N is the 80% percentile
# Write the resulting user id to germany_top (not the whole dataframe, just the id)
german_users = retail.loc[retail.Country == 'Germany'] \
        .groupby('CustomerID', as_index=False) \
        .agg({'InvoiceNo': 'nunique'})

In [103]:
percentil_80 = german_users.InvoiceNo.quantile(q=0.8)

In [105]:
germany_top = german_users.query('InvoiceNo > @percentil_80')['CustomerID']

In [108]:
# Leave data only for top German users
top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]

In [115]:
# Group top_retail_germany by StockCode
# Which of the products was added to the cart most often, except for POST?
top_retail_germany.groupby('StockCode', as_index=False).agg({'InvoiceNo': 'count'}).sort_values('InvoiceNo', ascending=False)

,StockCode,InvoiceNo
1167,POST,219
409,22326,62
452,22423,55
411,22328,45
474,22467,37
...,...,...
743,23008,1
739,22999,1
44,20717,1
333,22119,1


In [116]:
# Create a Revenue column with the purchase amount using the Quantity and UnitPrice columns
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [117]:
retail.Revenue

0         15.30
1         20.34
2         22.00
3         20.34
4         20.34
          ...  
541904    10.20
541905    12.60
541906    16.60
541907    16.60
541908    14.85
Name: Revenue, Length: 536641, dtype: float64

In [129]:
# For each transaction (InvoiceNo), calculate the final amount of the order
# Indicate the top 5 (InvoiceNo) by order amount (comma-separated with a space and in descending order of TotalRevenue)
print(*list(retail.groupby('InvoiceNo', as_index=False) \
        .agg({'Revenue': 'sum'}) \
        .rename(columns={'Revenue': 'TotalRevenue'}) \
        .sort_values('TotalRevenue', ascending=False) \
        .head(5)['InvoiceNo']), sep=', ')

581483, 541431, 574941, 576365, 556444
